**Loading Libraries**

In [1]:
import numpy as np
import pandas as pd
import pathlib
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf



**Pasting Image Directrory**

In [2]:
image_dir=Path('/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset')


# Creating File DataFrame

In [3]:
# Get filepaths and labels
filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)

# Drop GT images
image_df['Label'] = image_df['Label'].apply(lambda x: np.NaN if x[-2:] == 'GT' else x)
image_df = image_df.dropna(axis=0)

# Sample 200 images from each class
samples = []


for category in image_df['Label'].unique():
    category_slice = image_df.query("Label == @category")
    samples.append(category_slice.sample(200, random_state=1))

image_df = pd.concat(samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)



In [4]:
image_df

,Filepath,Label
0,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Gilt-Head Bream
1,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Sea Bass
2,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Red Mullet
3,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Shrimp
4,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Trout
...,...,...
1795,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Trout
1796,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Red Sea Bream
1797,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Striped Red Mullet
1798,/kaggle/input/a-large-scale-fish-dataset/Fish_...,Black Sea Sprat


In [5]:
train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

# **Loading Image Data**
# 

In [6]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [7]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 1008 validated image filenames belonging to 9 classes.
Found 252 validated image filenames belonging to 9 classes.
Found 540 validated image filenames belonging to 9 classes.


# **Load Pretrained Model**

In [8]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


# Training

In [9]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(9, activation='softmax')(x)


model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)  

Epoch 1/100


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1728208939.377595      93 service.cc:145] XLA service 0x7ddd90001c10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728208939.377662      93 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


 1/32 ━━━━━━━━━━━━━━━━━━━━ 7:28 14s/step - accuracy: 0.1562 - loss: 2.2461

I0000 00:00:1728208944.245625      93 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


32/32 ━━━━━━━━━━━━━━━━━━━━ 40s 811ms/step - accuracy: 0.6346 - loss: 1.2779 - val_accuracy: 0.9683 - val_loss: 0.1371
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 14s 359ms/step - accuracy: 0.9746 - loss: 0.0929 - val_accuracy: 0.9563 - val_loss: 0.1201
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 14s 350ms/step - accuracy: 0.9964 - loss: 0.0290 - val_accuracy: 0.9762 - val_loss: 0.0594
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 342ms/step - accuracy: 1.0000 - loss: 0.0099 - val_accuracy: 0.9722 - val_loss: 0.0649
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 355ms/step - accuracy: 1.0000 - loss: 0.0044 - val_accuracy: 0.9802 - val_loss: 0.0527
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 339ms/step - accuracy: 1.0000 - loss: 0.0033 - val_accuracy: 0.9802 - val_loss: 0.0553
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 348ms/step - accuracy: 1.0000 - loss: 0.0025 - val_accuracy: 0.9802 - val_loss: 0.0572
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 342ms/step - accuracy: 1.0000 - loss: 0.0018 - val_accuracy

# Result

In [11]:
results=model.evaluate(test_images, verbose=0)
print("Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}".format(results[1]*100))

Test Loss: 0.03272
Test Accuracy: 99.07
